## Code to record a single PUMP/probe experiment 
Experimental setup is here: https://www.protocols.io/edit/pump-probe-experiment-cbvxsn7n 

and in this paper: 

Gallot, T., et al. "Characterizing the nonlinear interaction of S-and P-waves in a rock sample." Journal of Applied Physics 117.3 (2015): 034902.

In [40]:
#Import necessary files etc
from numpy import *
from pylab import *
import string

from datetime import date

#for connecting with the scope
import AgilentSCPI
import time
import numpy as np
import sys
import matplotlib.pyplot as plt
import time

### Parameter setup

In [50]:
delayvec=np.arange(0,100,1)
n_aves=1024
#2 ms between pulses, this is how long to wait to ensure that one signal is 
#recorded before sending another signal
t_delay=3*n_aves*2e-3 
addtodelay=52  #shift so that probe waveform is in the centre of the oscilloscope screen at the first delay
#Place to write data (note we will make sub-directories with parameters)
fname_root="TS3_"  #prefix denoting experiment, keep to a few characters if saving to USB
ampval=5.0

### Classes for handling and storing the data

In [51]:
#For the data, store a time vector and data vector together
class timedata:

    def __init__(self, name):
        self.name = name
        self.time = np.array    # creates a new empty list
        self.data = np.array    # creates a new empty list


In [41]:
def readlabdat(fname,dataset):
# read in data from the scope and make a matrix of values from it
    
    
    f = open(fname,'r')
    tmp1=[]
    tmp2=[]
    
    for line in f:
        columns = line.split(',')
        tmp1.append(float(columns[0]))
        tmp2.append(float(columns[1]))

    dataset.time=np.asarray(tmp1)
    dataset.data=np.asarray(tmp2)
    f.close()
        

In [42]:
# read data from oscilloscope and save to file
def saveWave(filename,chs,npts=5000):
    print("Savewave 1")
    for ch in chs:
        print("Savewave 2")
        p1,x1 = OscRead2Numpy(osc,ch,npts)
#        p1,x1 = osc.OscRead2Numpy(ch,npts)

        if 'data' in locals():
            data = vstack((data,x1))
        else:
            data = x1
            
    print("Savewave 3")
    
    t = arange(len(x1))*p1[4] + p1[5]
    data = vstack((t,data)).T
    savetxt(filename, data, fmt="%12.6G", delimiter=',')
    print(str(shape(data)), " data points saved to "+filename)

### Setup Function Generator

In [46]:
funcgen = AgilentSCPI.SCPI("134.153.36.84")  #IP address is available from the equipment
#Check its status
q=funcgen.query(b'*IDN?\n')
print(q)
#q=fungen.query(bytes("*IDN?\n",'utf-8'))
#Load the setup file
setupFile="STATE_COSJ20.STA"
cmd = bytes(":MMEMory:LOAD:STATe 'INT:\\"+setupFile+"'\n",'utf-8')
funcgen.s.send(cmd)
#Set the amplitude of the PUMP (first) and then the probe
funcgen.s.send(bytes(":SOUR2:VOLT"+" "+str(ampval)+"\n",'utf-8'))
funcgen.s.send(bytes(":SOUR1:VOLT"+" 10.0"+"\n",'utf-8'))


b'Agilent Technologies,33510B,MY52202006,5.02-1.19-2.00-58-00\n'


17

### Setup Oscilloscope

In [47]:
#IP address available from scope in the LAN setup 
osc = AgilentSCPI.SCPI("134.153.36.224")
#Check the connection
q2=osc.query(b'*IDN?\n')
print(q2)
osc.reset()
#Set trigger parameters
osc.s.send(b":TRIGger:EDGE:SOURCe EXTernal\n")
osc.s.send(b":TRIGger:EDGE:LEVel 5.0\n")
osc.s.send(b":TRIGger:EDGE:SLOPe POSitive\n")
#Set time scale parameters
osc.s.send(b":TIM:REF LEFT\n")
osc.s.send(b":TIM:DEL 0us\n") #was set to 30 ... but why?
osc.s.send(b":TIM:SCAL 0.5us\n")
#Set voltage scale parameters
osc.s.send(b":CHANnel1:SCALe 0.01\n")
#Set acquisition and averaging parameters
osc.s.send(b":ACQuire:TYPE AVERage\n")
osc.s.send(b":ACQuire:COUNt 128\n")

b'AGILENT TECHNOLOGIES,MSO-X 2014A,MY54100349,02.65.2021030741\n'


19

### Run the experiment
First record the PUMP by itself, on a larger scale, then loop through all delays and collect all necessary signals.

In [48]:
#Set up the function generator so that only the PUMP is sent
funcgen.s.send(bytes(":SOUR2:VOLT"+" "+str(ampval)+"\n",'utf-8'))
funcgen.s.send(b":OUTP1 OFF\n")  #turn off probe
funcgen.s.send(b":OUTP2 ON\n")   #turn on pump
cmd=bytes(":TRIG:DEL "+str(0)+"us\n",'utf-8')  #set time delay to zero
funcgen.s.send(cmd)
#Save the PUMP waveform from the oscilloscope
osc.s.send(b":CHANnel2:DISPlay 2\n") #2= on 1=off  #turns on channel 2
osc.s.send(b":CHANnel1:DISPlay 0\n") # turns off channel 1

fname=fname_root+str("SB4.csv") 
#setup the time scale
osc.s.send(b":TIM:REF LEFT\n")  
osc.s.send(b":TIM:SCAL 20us\n")
osc.s.send(b":TIM:DEL 0us\n")
osc.s.send(b"CHANnel2:SCALe 50\n")
#saveWave(fname,[2])
#osc.s.send(b":TIM:SCAL 1.0us\n")
time.sleep(t_delay)
test=osc.OscSaveWave(fname)    
#osc.s.send(b":CHANnel2:DISPlay 0\n") #2= on 1=off
#osc.s.send(b":CHANnel1:DISPlay 2\n")

:SAVE:WAVeform 'TS1_SB4.csv'



In [49]:
osc.s.send(b":CHANnel2:DISPlay 0\n")
osc.s.send(b":CHANnel1:DISPlay 2\n")
osc.s.send(b":TIM:SCAL 1.0us\n")


scaldel=1  #parameter for changing the filename if there's a decimal in the delays
#Run through all of the delays and do the experiment
for delay in delayvec:
    #set delay
    cmd=":TRIG:DEL "+str(delay)+"us\n"  #shifts the signal from function generator
    funcgen.s.send(bytes(cmd,'utf-8'))
    
    #shift time scale on oscilloscope
    osc.s.send(bytes(":TIM:DEL "+ str(addtodelay+delay)+"us\n",'utf-8'))

    #record probe only
    funcgen.s.send(bytes(":OUTP1 ON\n",'utf-8')) #probe on
    funcgen.s.send(bytes(":OUTP2 OFF\n",'utf-8')) #PUMP off

    time.sleep(t_delay)

    fname=fname_root+"pr"+str(int(scaldel*delay))+".csv"
    osc.OscSaveWave(fname)
    
    time.sleep(t_delay)
    #saveWave(fname,[1])

    #record PUMP only
    osc.s.send(bytes(":TIM:DEL "+ str(addtodelay+delay)+"us\n",'utf-8'))
    funcgen.s.send(bytes(":OUTP1 OFF\n",'utf-8')) #probe off
    funcgen.s.send(bytes(":OUTP2 ON\n",'utf-8')) #PUMP on

    #funcgen.s.send(b":OUTP1 OFF\n") #probe on
    #funcgen.s.send(b":OUTP2 ON\n") #PUMP off
    time.sleep(t_delay)

    fname=fname_root+"pu"+str(int(scaldel*delay))+".csv"
    osc.OscSaveWave(fname)
    time.sleep(t_delay)
#    saveWave(fname,[1])

    #record PUMP and probe
    osc.s.send(bytes(":TIM:DEL "+ str(addtodelay+delay)+"us\n",'utf-8'))
    funcgen.s.send(bytes(":OUTP1 ON\n",'utf-8')) #probe on
    funcgen.s.send(bytes(":OUTP2 ON\n",'utf-8')) #PUMP off


    #funcgen.s.send(b":OUTP1 ON\n") #probe on
    #funcgen.s.send(b":OUTP2 ON\n") #PUMP off
    time.sleep(t_delay)

    fname=fname_root+"pp"+str(int(scaldel*delay))+".csv"
    osc.OscSaveWave(fname)
    time.sleep(t_delay)
   #saveWave(fname,[1])

#Turn off function generator so that we are not sending waves any longer
funcgen.s.send(b":OUTP1 OFF\n")
funcgen.s.send(b":OUTP2 OFF\n")
print(datetime.datetime.now())

:SAVE:WAVeform 'TS1_pr0.csv'

:SAVE:WAVeform 'TS1_pu0.csv'

:SAVE:WAVeform 'TS1_pp0.csv'

:SAVE:WAVeform 'TS1_pr1.csv'

:SAVE:WAVeform 'TS1_pu1.csv'

:SAVE:WAVeform 'TS1_pp1.csv'

:SAVE:WAVeform 'TS1_pr2.csv'

:SAVE:WAVeform 'TS1_pu2.csv'

:SAVE:WAVeform 'TS1_pp2.csv'

:SAVE:WAVeform 'TS1_pr3.csv'

:SAVE:WAVeform 'TS1_pu3.csv'

:SAVE:WAVeform 'TS1_pp3.csv'

:SAVE:WAVeform 'TS1_pr4.csv'

:SAVE:WAVeform 'TS1_pu4.csv'

:SAVE:WAVeform 'TS1_pp4.csv'

:SAVE:WAVeform 'TS1_pr5.csv'

:SAVE:WAVeform 'TS1_pu5.csv'

:SAVE:WAVeform 'TS1_pp5.csv'

:SAVE:WAVeform 'TS1_pr6.csv'

:SAVE:WAVeform 'TS1_pu6.csv'

:SAVE:WAVeform 'TS1_pp6.csv'

:SAVE:WAVeform 'TS1_pr7.csv'

:SAVE:WAVeform 'TS1_pu7.csv'

:SAVE:WAVeform 'TS1_pp7.csv'

:SAVE:WAVeform 'TS1_pr8.csv'

:SAVE:WAVeform 'TS1_pu8.csv'

:SAVE:WAVeform 'TS1_pp8.csv'

:SAVE:WAVeform 'TS1_pr9.csv'

:SAVE:WAVeform 'TS1_pu9.csv'

:SAVE:WAVeform 'TS1_pp9.csv'

:SAVE:WAVeform 'TS1_pr10.csv'

:SAVE:WAVeform 'TS1_pu10.csv'

:SAVE:WAVeform 'TS1_pp10.csv'

:SAVE:W

In [ ]:
funcgen.s.send(b":OUTP1 OFF\n")
funcgen.s.send(b":OUTP2 OFF\n")

In [ ]:
def OscRead2Numpy(osc,channel,npts=5000):
    
    print("hi 1")
    osc.s.send(b':WAVeform:POINts:MODE RAW\n')
    print("hi 2")

    osc.s.send(b':WAVeform:FORMat ASCII\n')
    print("hi 3")


    cmd = ":WAVeform:SOURce CHANnel"+str(channel)+"\n"
    osc.s.send(bytes(cmd,'utf-8'))
    print("hi 4")


    cmd = ":WAVeform:POINts "+str(npts)+"\n"
    osc.s.send(bytes(cmd,'utf-8'))
    print("hi 5")
    
    a=osc.s.send(b":WAVeform:PREamble?\n")
    print(a)
    osc.s.settimeout(100)
    print(osc.s.timeout)
    try:
        print("cow")
        c = osc.s.recv(1024)
        print(c)
    except osc.s.timeout:
        print("time out getting preamble")
    p = string.split(c, ",")
    p = [y.strip('+').rstrip("\n") for y in p]
    preamble = array(map(float,p))

    print("got preamble")



    return

In [ ]:
print(fname)
saveWave(fname,[2],npts=5)

In [ ]:
test=osc.OscSaveWave("Test1.csv")

In [ ]:
dat=np.loadtxt("/Volumes/Untitled/Test1.csv",skiprows=2,delimiter=',')

In [ ]:
np.shape(dat)
plt.plot(dat[:,0],dat[:,2])

In [ ]:
    self.s.send(b":WAVeform:PREamble?\n")
    try:
        c = self.s.recv(1024)
    except socket.timeout:
        print("time out getting preamble")
    p = string.split(c, ",")
    p = [y.strip('+').rstrip("\n") for y in p]
    preamble = array(map(float,p))

    print("got preamble")


    self.s.send(b':WAVeform:DATA?\n')
    skip = self.s.recv(10)
    buf = []
    self.s.settimeout(1)
    while True:
      try:
        data = self.s.recv(1024)
        buf.append(data)
      except socket.timeout:
        break

    self.s.settimeout(None)
    records = "".join(buf).split(',')
    records[len(records)-1] = records[len(records)-1].rstrip("\n")
    data = array(map(float, records))

    return preamble, data

In [18]:
delay=20
cmd=":TRIG:DEL "+str(delay)+"us\n"
funcgen.s.send(bytes(cmd,'utf-8'))
    

15

In [21]:
osc.s.send(bytes(":TIM:DEL "+ str(addtodelay+delay)+"us\n",'utf-8'))



14

In [11]:
fname='test.csv'
saveWave(fname,[1])

Savewave 1
Savewave 2


NameError: name 'socket' is not defined

In [10]:
    def OscRead2Numpy(self,channel,npts):
        self.s.send(b':WAVeform:POINts:MODE RAW\n')
        self.s.send(b':WAVeform:FORMat ASCII\n')
        cmd = ":WAVeform:SOURce CHANnel"+str(channel)+"\n"
        self.s.send(bytes(cmd,'utf-8'))

        cmd = ":WAVeform:POINts "+str(npts)+"\n"
        # print(cmd)
        self.s.send(bytes(cmd,'utf-8'))


        self.s.send(b":WAVeform:PREamble?\n")
        try:
            c = self.s.recv(1024)
        except socket.timeout:
            print("time out getting preamble")
        p = string.split(c, ",")
        p = [y.strip('+').rstrip("\n") for y in p]
        preamble = array(map(float,p))
        
        #print "got preamble"
     
     
        self.s.send(b':WAVeform:DATA?\n')
        skip = self.s.recv(10)
        buf = []
        self.s.settimeout(1)
        while True:
          try:
            data = self.s.recv(1024)
            buf.append(data)
          except socket.timeout:
            break

        self.s.settimeout(None)
        records = "".join(buf).split(',')
        records[len(records)-1] = records[len(records)-1].rstrip("\n")
        data = array(map(float, records))

        return preamble, data

